In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.


## Read Data 

In [2]:
def read_data(path, include_label):
    def scan_file():
        with open(path) as in_file:
            next(in_file)  # skip header
            for l in in_file:
                yield l.strip().split('\t')
    if include_label:
        return [(qid, did, q, d, float(label)) for qid, q, _, _, did, d, label in scan_file()]
    else:
        return [(qid, did, q, d) for qid, q, _, _, did, d, _ in scan_file()]

train = read_data('data/WikiQA/WikiQACorpus/WikiQA-train.tsv', include_label=True)
predict  = read_data('data/WikiQA/WikiQACorpus/WikiQA-test.tsv', include_label=False)

## Preprocess Data

In [3]:
train_pack = mz.pack(train[:1000])
predict_pack = mz.pack(predict[:1000])

In [4]:
preprocessor = mz.preprocessors.DSSMPreprocessor()
preprocessor.fit(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 96/96 [00:00<00:00, 1163.79it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 1000/1000 [00:00<00:00, 3588.42it/s]
Fitting vocabulary unit.: 100%|██████████| 84935/84935 [00:00<00:00, 2623312.47it/s]


In [5]:
train_pack_processed = preprocessor.transform(train_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 96/96 [00:00<00:00, 3666.88it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 1000/1000 [00:00<00:00, 2734.69it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 108/108 [00:00<00:00, 5574.58it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 992/992 [00:00<00:00, 3121.28it/s]


## Generate Data

In [6]:
train_generator = mz.DataGenerator(train_pack_processed, batch_size=32)

## Model Building

In [7]:
ranking_task = mz.tasks.Ranking()
ranking_task.metrics = [
    'mae', 'map', 'precision',
    mz.metrics.Precision(k=3),
    mz.metrics.DiscountedCumulativeGain(k=1),
    mz.metrics.DiscountedCumulativeGain(k=3),
    mz.metrics.DiscountedCumulativeGain(k=5),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=1),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5)
]
ranking_task.metrics

['mae',
 mean_average_precision(0),
 precision@1(0),
 precision@3(0),
 discounted_cumulative_gain@1(0),
 discounted_cumulative_gain@3(0),
 discounted_cumulative_gain@5(0),
 normalized_discounted_cumulative_gain@1(0),
 normalized_discounted_cumulative_gain@3(0),
 normalized_discounted_cumulative_gain@5(0)]

In [8]:
model = mz.models.DSSMModel()
model.params['task'] = ranking_task
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.guess_and_fill_missing_params()
model.build()
model.compile()

Parameter "name" set to DSSMModel.


## Model Training

In [9]:
model.fit(*train_pack_processed.unpack(), epochs=2, batch_size=32)

Epoch 1/2
1000/1000 [==============================] - 1s 760us/step - loss: 0.2165 - mean_absolute_error: 0.3485
Epoch 2/2
1000/1000 [==============================] - 0s 486us/step - loss: 0.1675 - mean_absolute_error: 0.3225


In [10]:
model.fit_generator(train_generator, epochs=2, workers=4, use_multiprocessing=True)

Epoch 1/2
32/32 [==============================] - 1s 24ms/step - loss: 0.0729 - mean_absolute_error: 0.2057
Epoch 2/2
32/32 [==============================] - 1s 22ms/step - loss: 0.0352 - mean_absolute_error: 0.1404


### Training with Dynamic Data Generator

In [11]:
pp_no_hashing = mz.preprocessors.DSSMPreprocessor(with_word_hashing=False)
train_pack_processed_no_hashing = pp_no_hashing.fit_transform(train_pack)
word_hashing_unit = mz.processor_units.WordHashingUnit(pp_no_hashing.context['vocab_unit'].state['term_index'])
dynamic_data_generator = mz.data_generator.DynamicDataGenerator(word_hashing_unit.transform,
                                                                train_pack_processed_no_hashing)
model.fit_generator(dynamic_data_generator, epochs=2, workers=4, use_multiprocessing=True)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 96/96 [00:00<00:00, 4648.93it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 1000/1000 [00:00<00:00, 3631.78it/s]
Fitting vocabulary unit.: 100%|██████████| 84935/84935 [00:00<00:00, 2614686.64it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 96/96 [00:00<00:00, 6228.49it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 1000/1000 [00:00<00:00, 3825.02it/s]


Epoch 1/2
32/32 [==============================] - 1s 22ms/step - loss: 0.0201 - mean_absolute_error: 0.1027
Epoch 2/2
Epoch 1/2
32/32 [==============================] - 1s 26ms/step - loss: 0.0123 - mean_absolute_error: 0.0757


## Model Predicting 

In [12]:
model.predict(predict_pack_processed[:10].unpack()[0])

array([[-0.4900815 ],
       [-0.45969403],
       [-0.50754285],
       [-0.53743637],
       [-0.3937044 ],
       [-0.713043  ],
       [-1.2054659 ],
       [-1.1665266 ],
       [-1.1776927 ],
       [-0.94183373]], dtype=float32)

## Model Evaluating

In [13]:
model.evaluate(*train_pack_processed.unpack())

1000/1000 [==============================] - 0s 129us/step


{'loss': 0.007685753099620342,
 'mean_absolute_error': 0.06394205537438392,
 'mean_average_precision(0)': 0.3333333333333333,
 'precision@1(0)': 0.3333333333333333,
 'precision@3(0)': 0.13194444444444445,
 'discounted_cumulative_gain@1(0)': 0.4808983469629878,
 'discounted_cumulative_gain@3(0)': 0.5338530547483661,
 'discounted_cumulative_gain@5(0)': 0.5338530547483661,
 'normalized_discounted_cumulative_gain@1(0)': 0.3333333333333333,
 'normalized_discounted_cumulative_gain@3(0)': 0.3333333333333333,
 'normalized_discounted_cumulative_gain@5(0)': 0.3333333333333333}

## Automation

In [14]:
train_pack.shuffle(inplace=True)

In [15]:
divider = int(len(train_pack) * 0.9)
train_pack_divided = train_pack[:divider]
test_pack_divided = train_pack[divider:]

In [16]:
director = mz.Director()
director.params['models'] = [
    mz.models.DenseBaselineModel(),
    mz.models.DSSMModel(),
    mz.models.ArcIModel()
]
director.params['evals_per_model'] = 5
director.params['train_pack'] = train_pack_divided
director.params['test_pack'] = test_pack_divided
director.params['task'] = ranking_task

In [ ]:
all_trials = director.action()

tpe_transform took 0.000735 seconds
TPE using 0 trials
tpe_transform took 0.000894 seconds
TPE using 1/1 trials with best loss 6598.083984
tpe_transform took 0.000859 seconds
TPE using 2/2 trials with best loss 6598.083984
tpe_transform took 0.000720 seconds
TPE using 3/3 trials with best loss 2988.430664


In [ ]:
for model_trials in all_trials:
    for trial in model_trials:
        print('Model Name:', trial['model_params']['name'])
        print("Hyper Parameter Sampled:" , trial['sampled_params'])
        print('Loss:', trial['loss'])
        print()

## Misc

### Load Embedding Matrix

In [ ]:
model = mz.models.ArcIModel()
model.params['task'] = ranking_task
term_index = preprocessor.context['vocab_unit'].state['term_index']
model.params['embedding_input_dim'] = len(term_index) + 1
model.guess_and_fill_missing_params()
model.build()
model.compile()

In [ ]:
vocab = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = np.random.uniform(
    -0.2, 0.2,
    (model.params['embedding_input_dim'], model.params['embedding_output_dim'])
)
model.load_embedding_matrix(embedding_matrix)